<div class="alert alert-block alert-info"> <b>NOTE</b> Please select the kernel <code>Python [conda env: deepcell]</code> for this notebook. </div>

# 4. Evaluate

In [10]:
import glob
import os
import re
import subprocess

import numpy as np
import pandas as pd

from deepcell_tracking.metrics import TrackingMetrics

Some conversion from the gnn output to isbi compatible format may be necessary

In [11]:
data_dir = '/publication-tracking/benchmarking/DeepCell/data'
gt_seg_dir = os.path.join(data_dir, 'SEG_GT')
pred_seg_dir = os.path.join(data_dir, 'SEG_PRED')

pattern = re.compile('\d{3}_GT')
data_ids = [f.split('_')[0] for f in os.listdir(gt_seg_dir) if pattern.fullmatch(f)]

node_match_threshold = 0.6

ctc_software = '/publication-tracking/benchmarking/CTC_Evaluation_Software'
operating_system = 'Linux' # or 'Mac' or 'Win'
num_digits = '3'

In [12]:
benchmarks = []

for results_dir, s in zip([gt_seg_dir, pred_seg_dir], ['GT', 'Deepcell']):
    for data_id in data_ids:
        results = {
            'model': f'Deepcell - {s}',
            'data_id': os.path.splitext(data_id)[0]
        }
        gt_dir = os.path.join(results_dir, f'{data_id}_GT/TRA')
        res_dir = os.path.join(results_dir, f'{data_id}_RES')
        
        # Deepcell benchmarking
        m = TrackingMetrics.from_isbi_dirs(gt_dir, res_dir, threshold=node_match_threshold)
        results.update(m.stats)
        
        # CTC metrics
        for metric, path in [('DET', 'DETMeasure'), ('SEG', 'SEGMeasure'), ('TRA', 'TRAMeasure')]:
            p = subprocess.run([os.path.join(ctc_software, operating_system, path), results_dir, data_id, num_digits],
                               stdout=subprocess.PIPE)
            outstring = p.stdout
            
            try:
                val = float(outstring.decode('utf-8').split()[-1])
                results[metric] = val
            except:
                print('Benchmarking failure', path, results_dir, data_id)
                print(outstring.decode('utf-8'))
        
        benchmarks.append(results)

df = pd.DataFrame(benchmarks)
df.to_csv('benchmarks.csv')

missed node 1_29 division completely
missed node 29_29 division completely
missed node 57_10 division completely
missed node 60_0 division completely
18_16 out degree = 2, daughters mismatch, gt and res degree equal.
missed node 5_6 division completely
missed node 121_42 division completely
missed node 144_28 division completely
missed node 26_25 division completely
missed node 1_29 division completely
missed node 3_23 division completely
29_29 out degree = 2, daughters mismatch, gt and res degree equal.
corrected division 3_23 as a frameshift division not an error
missed node 10_19 division completely
15_17 out degree = 1, daughters mismatch.
missed node 19_20 division completely
29_17 out degree = 1, daughters mismatch.
missed node 60_34 division completely
missed node 76_7 division completely
corrected division 10_19 as a frameshift division not an error
corrected division 60_34 as a frameshift division not an error
corrected division 76_7 as a frameshift division not an error
misse